<a href="https://colab.research.google.com/github/haribharadwaj/notebooks/blob/main/BME511/PermutationTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Permutation tests

Permutation tests are powerful in that only minimal distributional assumptions need to be made as long as **exchangeability** is satisfied.

Note that exchangeable random variables are ones whose joint distribution doesn't change when we permute their ordering. This requirement is satisfied in most experimental scenarios under $\mathcal{H}_0$.

## Comparison of conventional statistical testing with permutation testing

Here, let's do conventional statistical testing (e.g., assuming data is normal) and see the impact of deviations from assumptions. Let's then see if permutation tests are susceptible to similar issues.

In particular, we will examine the effect of a heavier-than-normal tail (subexponential).

In [1]:
import numpy as np
from scipy import stats
import pylab as pl

In [2]:
# Draw two groups of normal measurements, with 20 samples each, but with the same distribution

p_val_cutoff = 0.05
Nreps = 1000
p = np.zeros(Nreps)
for k in range(Nreps):
    N = 20
    x1 = np.random.randn(N)
    x2 = np.random.randn(N)
    t, p[k] = stats.ttest_ind(x1, x2)

print(f'The proportion of tests with false positives = {(p < p_val_cutoff).sum()/Nreps}')

The proportion of tests with false positives = 0.052


In [3]:
# Repeat the same, but with a heavy tailed distribution.
# Here, we will use a cauchy distribution

p_val_cutoff = 0.05
Nreps = 1000
p = np.zeros(Nreps)
for k in range(Nreps):
    N = 20
    x1 = stats.cauchy.rvs(0, 1, N)
    x2 = stats.cauchy.rvs(0, 1, N)
    t, p[k] = stats.ttest_ind(x1, x2)

print(f'The proportion of tests with false positives = {(p < p_val_cutoff).sum()/Nreps}')

The proportion of tests with false positives = 0.02


As we can see, the t-test is too conservative in this case!

## Same experiments but with permutation testing

Let's start by writing a function that can do permutation tests with a given number of permutations.

In [4]:
def permtest(x1, x2, Nperms):
    N1 = x1.shape[0]
    N2 = x2.shape[0]
    perms = np.zeros(Nperms)
    bucket = np.concatenate([x1, x2])
    for k in range(Nperms):
        x = np.random.permutation(bucket)
        x1perm = x[:N1]
        x2perm = x[N1:]
        perms[k] = x1perm.mean() - x2perm.mean()
    actual = x1.mean() - x2.mean()
    p = np.float64((np.abs(perms) > np.abs(actual)).sum()) / Nperms
    return p    

In [5]:
# Draw two groups of normal measurements, with 20 samples each, but with the same distribution
p_val_cutoff = 0.05
Nreps = 1000
p = np.zeros(Nreps)
for k in range(Nreps):
    N = 20
    x1 = np.random.randn(N)
    x2 = np.random.randn(N)
    p[k] = permtest(x1, x2, 100)

print(f'The proportion of tests with false positives = {(p < p_val_cutoff).sum()/Nreps}')

The proportion of tests with false positives = 0.05


In [6]:
# Repeat the same, but with a heavy tailed distribution.
# Here, we will use a cauchy distribution

p_val_cutoff = 0.05
Nreps = 1000
p = np.zeros(Nreps)
for k in range(Nreps):
    N = 20
    x1 = stats.cauchy.rvs(0, 1, N)
    x2 = stats.cauchy.rvs(0, 1, N)
    p[k] = permtest(x1, x2, 100)

print(f'The proportion of tests with false positives = {(p < p_val_cutoff).sum()/Nreps}')

The proportion of tests with false positives = 0.045
